# PGO demo

This is a simple demonstration of how to use Profile-Guided Optimization (PGO) with golang on Kubernetes.

## Prerequisites

- A linode API key as `TF_VAR_linode_token` environment variable.
  - I recommend using [direnv](https://direnv.net/) to manage environment variables with the vscode extension.
- A container registry

In [ ]:
# Set registry variable
registry = 'ghcr.io/guilhem/pgo-demo'

# ensure TF_VAR_linode_token is set
import os  # noqa: E402
if 'TF_VAR_linode_token' not in os.environ:
    print('Please set the TF_VAR_linode_token environment variable')
    exit(1)

# set KUBECONFIG to $PWD/kubeconfig.yaml
os.environ['KUBECONFIG'] = os.path.join(os.getcwd(), 'kubeconfig.yaml')

## Instanciate a Linode Kubernetes cluster

```mermaid
flowchart TD
    A[Kubernetes cluster]
```

In [ ]:
from IPython.display import Code

Code(filename='cluster.tf', language='terraform')

In [ ]:
%%bash

terraform apply -auto-approve

In [ ]:
from IPython.display import Code

Code(filename='kubeconfig.yaml', language='yaml')

In [ ]:
%%bash
kubectl config view

## Deploy parca application

parca is a tool to analyze and visualize profiles. It can be used to analyze profiles generated by PGO.

It runs as a web application and can be deployed on Kubernetes.

```mermaid
flowchart TD
    B[Parca UI] --> A
    C[Parca Agent] --> A
    A[Kubernetes cluster]
```

In [ ]:
%%bash

kubectl apply -f https://github.com/parca-dev/parca/releases/download/v0.22.0/kubernetes-manifest.yaml

Parca also need an agent to collect profiles with ebpf as a daemonset.

In [ ]:
%%bash

kubectl apply -f https://github.com/parca-dev/parca-agent/releases/download/v0.33.1/kubernetes-manifest.yaml

In [ ]:
%%bash

kubectl get pods -n parca -o wide -w

We can open a port-forward to access the parca web application.

We are creating a persistant port-forward to access the parca web application on localhost:7070.

In [ ]:
import os
get_ipython().system = os.system # type: ignore
!kubectl -n parca port-forward service/parca 7070 & echo $! > /tmp/parca-forward.pid

In [ ]:
!kill $(cat /tmp/parca-forward.pid)

We can now access our parca web application on [localhost:7070][parcaUrl].

[parcaUrl]: http://localhost:7070

```mermaid
flowchart TD
    E[Browser] -- :7070--> D
    D[port-forward] --> B
    B[Parca UI] --> A
    C[Parca Agent] --> A
    A[Kubernetes cluster]
```

## Demo application

We will deploy a simple golang application that generates profiles.
This application render a markdown file, send with HTTP request into html.

In [ ]:
%%bash
cd demo; go build -o demo .

## Create a docker image

In [ ]:
from IPython.display import Code

Code(filename='demo/Dockerfile', language='Docker')

In [ ]:
%%bash -s "$registry"
docker build --platform linux/amd64 -t $1:nopgo -f demo/Dockerfile --push demo/

## Run our application in the cluster

In this notebook, we will deploy our application in the cluster. We will use the `kubectl run` command to deploy our application.

```mermaid
flowchart TD
    E[Browser] -- :7070--> D
    D[port-forward] --> B
    B[Parca UI] --> A
    C[Parca Agent] --> A
    A[Kubernetes cluster]
    F[demo app] --> A
```

In [ ]:
%%bash -s "$registry"
kubectl run pgo-demo --image=$1:nopgo --port=8080

## Test our application

Get an example markdown file

In [ ]:

! curl -o test.md -L "https://raw.githubusercontent.com/golang/go/c16c2c49e2fa98ae551fc6335215fadd62d33542/README.md"
from IPython.display import Code

Code(filename='test.md', language='Markdown')

## Test Markdown rendering

```mermaid
flowchart TD
    E[Browser] -- :7070--> D
    E -- :8080 --> G
    D[port-forward] --> B
    B[Parca UI] --> A
    C[Parca Agent] --> A
    A[Kubernetes cluster]
    F[demo app] --> A
    G[port-forward] --> F
```

In [ ]:
%%bash
kill $(cat port-forward.pid) || true
kubectl port-forward pods/pgo-demo 8080 & echo $! > port-forward.pid
sleep 1
curl --data-binary @test.md http://localhost:8080/render

kill $(cat port-forward.pid)

## Load data in application

```mermaid
flowchart TD
    E[Browser] -- :7070--> D
    E -- :8080 --> G
    D[port-forward] --> B
    B[Parca UI] --> A
    C[Parca Agent] --> A
    A[Kubernetes cluster]
    F[demo app] --> A
    G[port-forward] --> F
    H[load] --> G
```

In [ ]:
%%bash
kill $(cat port-forward.pid) || true
kubectl port-forward pods/pgo-demo 8080 & echo $! > port-forward.pid
sleep 2
go run github.com/prattmic/markdown-pgo/load@latest -count 1000 -source test.md & echo $! > load1.pid
go run github.com/prattmic/markdown-pgo/load@latest -count 1000 -source test.md & echo $! > load2.pid
go run github.com/prattmic/markdown-pgo/load@latest -count 1000 -source test.md & echo $! > load3.pid

wait $(cat load1.pid) $(cat load2.pid) $(cat load3.pid)

kill $(cat port-forward.pid)  || true

## See profile data in parca and generate a ppof file

```mermaid
flowchart TD
    E[Browser] -- :7070--> D
    E -- :8080 --> G
    D[port-forward] --> B
    B[Parca UI] --> A
    C[Parca Agent] --> A
    F -- eBPF --> C
    A[Kubernetes cluster]
    F[demo app] --> A
    G[port-forward] --> F
    H[load] --> G
```

[Parca on default namespace][parcaUrl]

[parcaUrl]: http://localhost:7070

After generating the ppof file, we can download it and use it to build our application with PGO.

In [ ]:
%%bash
cd demo/
mv ~/Downloads/profile.pb.gz ./profile.pb.gz
gunzip -f profile.pb.gz

## Compile with PGO activated

-pgo flag to enable PGO
[https://go.dev/doc/pgo](https://go.dev/doc/pgo)

In [ ]:
%%bash
cd demo; go build -o demo-pgo -pgo ./profile.pb .

## Create a new docker image

In [ ]:
from IPython.display import Code

Code(filename='demo/Dockerfile.pgo', language='Docker')

In [ ]:
%%bash -s "$registry"
docker build --platform linux/amd64 -t $1:pgo -f demo/Dockerfile.pgo --push demo/

In [ ]:
%%bash -s "$registry"
kubectl run pgo-demo-pgo --image=$1:pgo --port=8080

```mermaid
flowchart TD
    E[Browser] -- :7070--> D
    E -- :8080 --> G
    D[port-forward] --> B
    B[Parca UI] --> A
    C[Parca Agent] --> A
    A[Kubernetes cluster]
    F[demo app] --> A
    I[demo app with pgo] --> A
    G[port-forward] --> F
    H[load] --> G
```

## Benchmark our application

Run 2 benchmarks with the same markdown file

In [ ]:
%%bash
kill $(cat port-forward.pid) $(cat port-forward-pgo.pid) || true
kubectl port-forward pods/pgo-demo 8080 & echo $! > port-forward.pid
kubectl port-forward pods/pgo-demo-pgo 8081:8080 & echo $! > port-forward-pgo.pid
go test github.com/prattmic/markdown-pgo/load -bench=. -count=40 -source $(pwd)/test.md -addr "http://localhost:8080" > nopgo.txt & echo $! > bench-nopgo.pid
go test github.com/prattmic/markdown-pgo/load -bench=. -count=40 -source $(pwd)/test.md -addr "http://localhost:8081" > withpgo.txt & echo $! > bench-pgo.pid

wait $(cat bench-nopgo.pid) $(cat bench-pgo.pid)

kill  $(cat port-forward-pgo.pid) $(cat port-forward.pid)

results

In [ ]:
from IPython.display import Code

Code(filename='nopgo.txt')
Code(filename='withpgo.txt')

Compare the results

In [ ]:
!go install golang.org/x/perf/cmd/benchstat@latest

In [ ]:
%%bash
benchstat nopgo.txt withpgo.txt

## destroy the cluster

we can now destroy the cluster

In [ ]:
!terraform destroy -auto-approve

## Test locally

We can also test our application locally

```mermaid
flowchart TD
    A[load] --> C
    B[load] --> D
    C[app]
    D[app with pgo]
```

In [ ]:
%%bash
# run demo/demo and demo/demo-pgo in background
./demo/demo -port 8086 & echo $! > demo.pid
./demo/demo-pgo -port 8085 & echo $! > demo-pgo.pid

sleep 1

# run benchmark in background
go test github.com/prattmic/markdown-pgo/load -bench=. -count=40 -source $(pwd)/test.md -addr http://localhost:8086 > nopgo-local.txt & echo $! > nopgo-local.pid
go test github.com/prattmic/markdown-pgo/load -bench=. -count=40 -source $(pwd)/test.md -addr http://localhost:8085 > withpgo-local.txt & echo $! > withpgo-local.pid

# wait for the benchmark to finish
wait $(cat nopgo-local.pid) $(cat withpgo-local.pid)

# kill
kill $(cat demo.pid) $(cat demo-pgo.pid)

In [ ]:
%%bash
benchstat nopgo-local.txt withpgo-local.txt